# PyCaret Multiclass Classification — Kaggle
This notebook is Kaggle-ready for multiclass classification.

### Steps to use:
1. In Kaggle, click **Add Data** and attach a dataset (e.g., the Iris dataset).
2. Run all cells step-by-step.
3. Choose your dataset CSV from `/kaggle/input` and specify the target column (e.g., `species`).
4. PyCaret will automatically compare models, show results, and save the best model.

Assignment Done by :- **Dev Mulchandani**

In [1]:
%pip -q install -U pip setuptools wheel
%pip -q install 'pycaret==3.3.2'
import sys
print('✅ Python version:', sys.version)
print('✅ PyCaret installed successfully!')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.0 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.2 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
preprocessing 0.1.

In [2]:
import glob, pandas as pd
csvs = sorted([p for p in glob.glob('/kaggle/input/**/*.csv', recursive=True)])
if not csvs:
    raise SystemExit('❌ No CSVs found under /kaggle/input. Attach a dataset first.')
for i, p in enumerate(csvs):
    print(f'{i}: {p}')
idx = int(input('Enter the index of the CSV to load: '))
DATA_PATH = csvs[idx]
print('✅ Using:', DATA_PATH)
data = pd.read_csv(DATA_PATH)
print('📊 Shape:', data.shape)
display(data.head())

0: /kaggle/input/iris-train/Iris.csv


Enter the index of the CSV to load:  0


✅ Using: /kaggle/input/iris-train/Iris.csv
📊 Shape: (150, 6)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
# --- Force CPU mode & avoid RAPIDS imports (must run BEFORE importing PyCaret) ---
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # hide GPUs so libraries skip GPU paths
# If any RAPIDS packages are present, remove them so PyCaret won't try GPU engines
!pip -q uninstall -y cuml cudf cuml-cu12 cudf-cu12 cupy-cuda12x cupy-cuda11x cupy || true


In [4]:
from pycaret.classification import *
TARGET = input('Enter the multiclass target column name: ')
exp = setup(data, target=TARGET, session_id=123, use_gpu=False)
best = compare_models()
evaluate_model(best)
save_model(best, 'best_multiclass_model')

Enter the multiclass target column name:  Species


,Description,Value
0,Session id,123
1,Target,Species
2,Target type,Multiclass
3,Target mapping,"Iris-setosa: 0, Iris-versicolor: 1, Iris-virginica: 2"
4,Original data shape,"(150, 6)"
5,Transformed data shape,"(150, 6)"
6,Transformed train set shape,"(105, 6)"
7,Transformed test set shape,"(45, 6)"
8,Numeric features,5
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.7410
knn,K Neighbors Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0280
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0170
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1250
ada,Ada Boost Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0570
gbc,Gradient Boosting Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1080
lightgbm,Light Gradient Boosting Machine,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,7.2620
nb,Naive Bayes,0.9909,1.0000,0.9909,0.9927,0.9908,0.9862,0.9873,0.0170
qda,Quadratic Discriminant Analysis,0.9909,0.0000,0.9909,0.9927,0.9908,0.9862,0.9873,0.0180
lda,Linear Discriminant Analysis,0.9909,0.0000,0.9909,0.9927,0.9908,0.9862,0.9873,0.0160


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Id', 'SepalLengthCm',
                                              'SepalWidthCm', 'PetalLengthCm',
                                              'PetalWidthCm'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,...
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                          

In [5]:
# 🔮 Predict with the saved multiclass model (robust path picker)
from pycaret.classification import load_model, predict_model
import pandas as pd, glob

# 1) Load your trained model
model = load_model('best_multiclass_model')

# 2) List all CSVs under /kaggle/input and choose by index
csvs = sorted(glob.glob('/kaggle/input/**/*.csv', recursive=True))
if not csvs:
    raise SystemExit("No CSVs found. Click 'Add Data' in Kaggle and attach a dataset, then rerun.")
for i, p in enumerate(csvs):
    print(f"{i}: {p}")

idx = int(input("Enter the index of the CSV to use for predictions: "))
test_path = csvs[idx]
print("✅ Using:", test_path)

# 3) Read and predict
test = pd.read_csv(test_path)
preds = predict_model(model, data=test)

# 4) Show a preview
preds.head()


Transformation Pipeline and Model Successfully Loaded
0: /kaggle/input/iris-train/Iris.csv


Enter the index of the CSV to use for predictions:  0


✅ Using: /kaggle/input/iris-train/Iris.csv


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.9933,1.0000,0.9933,0.9935,0.9933,0.9900,0.9901


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,prediction_label,prediction_score
0,1,5.1,3.5,1.4,0.2,Iris-setosa,Iris-setosa,1.0
1,2,4.9,3.0,1.4,0.2,Iris-setosa,Iris-setosa,1.0
2,3,4.7,3.2,1.3,0.2,Iris-setosa,Iris-setosa,1.0
3,4,4.6,3.1,1.5,0.2,Iris-setosa,Iris-setosa,1.0
4,5,5.0,3.6,1.4,0.2,Iris-setosa,Iris-setosa,1.0


In [6]:
# Save predictions to CSV
preds[['Id', 'Species', 'prediction_label']].to_csv('predictions.csv', index=False)
print("✅ Predictions saved to predictions.csv")


✅ Predictions saved to predictions.csv
